<a href="https://colab.research.google.com/github/ToshiroHJJZ/Detection-and-Classification-of-Linguistic-Features-for-Effective-Spam-Identification/blob/main/Adversarial_Attack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
# Install necessary libraries
!pip install textattack
!pip install transformers # install the transformers library

In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import textattack
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
import os

In [18]:
# encountered a utf-8 error,UnicodeDecodeError: 'utf-8' codec can't decode bytes in position 606-607:
#invalid continuation byte, encoding pandas to use 'latin-1', or 'ISO-8859-1'
df = pd.read_csv("/content/drive/MyDrive/spam.csv", encoding='latin-1')

In [19]:
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [20]:
df=df.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], axis=1)

In [21]:
x=df['v2']
y=df['v1']

In [22]:
# label transform v1
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
y=le.fit_transform(y)

In [23]:
y[:5]

array([0, 0, 1, 0, 0])

In [24]:
# Specify the model name and revision
model_name = "distilbert-base-uncased-finetuned-sst-2-english"
revision = "af0f99b"

# Load the model with the specified name and revision
# Instead of using pipeline, load the model directly
model = AutoModelForSequenceClassification.from_pretrained(model_name, revision=revision)
tokenizer = AutoTokenizer.from_pretrained(model_name, revision=revision)

# Wrap the model using the HuggingFaceModelWrapper
model_wrapper = textattack.models.wrappers.HuggingFaceModelWrapper(model, tokenizer)

# Initialize the attack with the wrapped model
attack = textattack.attack_recipes.TextFoolerJin2019.build(model_wrapper)


# Generate adversarial examples for the first 50 rows
original_texts = df['v2'][:160]  # Assuming 'df' is your DataFrame
adversarial_texts = []
successfully_attacked = []

for text in original_texts:
    try:
        attack_result = attack.attack([text])
        adversarial_texts.append(attack_result[0].perturbed_text())
        successfully_attacked.append(True)
    except Exception as e:
        print(f"Failed to attack: {text}")
        adversarial_texts.append(text)
        successfully_attacked.append(False)

# Combine original and adversarial examples
combined_texts = list(original_texts) + adversarial_texts
labels = [0] * len(original_texts) + [1] * len(adversarial_texts)  # 0 = non-adversarial, 1 = adversarial

# Create a DataFrame with the combined data
df_combined = pd.DataFrame({
    'text': combined_texts,
    'label': labels
})

# Specify the output folder path
output_folder = r"C:\Users\jonat\Desktop\Thesis papers\Thesis SPAM"

# Create the output folder if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Construct the full file path
file_path = os.path.join(output_folder, 'adversarial_social_media_dataset.csv')

# Save the dataset to the specified folder
df_combined.to_csv(file_path, index=False)
print(f"Dataset saved successfully to: {file_path}")

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
textattack: Unknown if model of class <class 'transformers.models.distilbert.modeling_distilbert.DistilBertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


Failed to attack: Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...
Failed to attack: Ok lar... Joking wif u oni...
Failed to attack: Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's
Failed to attack: U dun say so early hor... U c already then say...
Failed to attack: Nah I don't think he goes to usf, he lives around here though
Failed to attack: FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, å£1.50 to rcv
Failed to attack: Even my brother is not like to speak with me. They treat me like aids patent.
Failed to attack: As per your request 'Melle Melle (Oru Minnaminunginte Nurungu Vettam)' has been set as your callertune for all Callers. Press *9 to copy your friends Callertune
Failed to attack: WINNER!! As a valued network customer yo